[View in Colaboratory](https://colab.research.google.com/github/ysterin/deep-pointing/blob/master/deep_pointing_tensorflow.ipynb)

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [19]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1aOdHBIjB0fOLg9sQMhOlZI6gy_oMPHty'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
{ord(c) for c in text}

{32,
 1456,
 1457,
 1458,
 1459,
 1460,
 1461,
 1462,
 1463,
 1464,
 1465,
 1466,
 1467,
 1468,
 1473,
 1474,
 1488,
 1489,
 1490,
 1491,
 1492,
 1493,
 1494,
 1495,
 1496,
 1497,
 1498,
 1499,
 1500,
 1501,
 1502,
 1503,
 1504,
 1505,
 1506,
 1507,
 1508,
 1509,
 1510,
 1511,
 1512,
 1513,
 1514}

In [36]:
import numpy as np
from collections import Counter
import re
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [221]:
def tensorize(text):
  unpointed_text = [c for c in text if ord(c) in range(1488, 1515) or c==' '][0:]
  pointing_seqs = re.split(r'\w| ', text)[1:]
  print(pointing_seqs[:100])
  print(unpointed_text[:100])
  
  assert len(unpointed_text) == len(pointing_seqs)
  N = len(unpointed_text)
  text_ = ''
  for i in range(N):
    text_ += unpointed_text[i] + pointing_seqs[i]
  assert text_==text
  letters_index = { chr(i):i-1487 for i in range(1488, 1515)}
  letters_index[' '] = 0
  
  X = np.asarray([letters_index[c] for c in unpointed_text])
  
  pointings = [chr(i) for i in range(1456, 1468)]
  diphthongs = [''.join([chr(i) for i in l]) for l in [[1464, 1460], [1463, 1456], [1464, 1456], [1463, 1460]]]
  num_pointings = len(pointings)
  dagesh = chr(1468)
  shin = chr(1473)
  sin = chr(1474)
  
  ys = [np.zeros(N, dtype=np.int32) for _ in range(3)]
  
  for i, seq in enumerate(pointing_seqs):
    if dagesh in seq:
      ys[0][i] = 1
    if shin in seq:
      ys[1][i] = 1
    elif sin in seq:
      ys[1][i] = 2
    
    flag = False
    for j, diph in enumerate(diphthongs):
      if diph in seq:
        ys[2][i] = j + num_pointings + 1
        flag = True
        break
    if flag:
      continue
      
    for j, point in enumerate(pointings):
      if point in seq:
        if ys[2][i] != 0:
          print(point)
        
        ys[2][i] = j + 1
        
  return X, ys
  
X, ys = tensorize(text)
print(X[:100])
ys[2][:120]

['', 'ְּ', 'ֵ', '', 'ִׁ', '', '', '', 'ָּ', 'ָ', '', '', 'ֱ', 'ֹ', 'ִ', '', '', '', 'ֵ', '', '', 'ַ', 'ָּׁ', 'ַ', 'ִ', '', '', 'ְ', 'ֵ', '', '', 'ָ', 'ָ', 'ֶ', '', '', 'ְ', 'ָ', 'ָ', 'ֶ', '', '', 'ָ', 'ְ', 'ָ', '', '', 'ֹ', '', 'ּ', '', 'ָ', 'ֹ', '', 'ּ', '', 'ְ', 'ֹ', 'ֶׁ', 'ְ', '', 'ַ', '', '', 'ְּ', 'ֵ', '', '', 'ְ', '', 'ֹ', '', '', 'ְ', '', 'ּ', 'ַ', '', 'ֱ', 'ֹ', 'ִ', '', '', '', 'ְ', 'ַ', 'ֶ', 'ֶ', '', '', 'ַ', '', '', 'ְּ', 'ֵ', '', '', 'ַ', 'ָּ', 'ִ']
[' ', 'ב', 'ר', 'א', 'ש', 'י', 'ת', ' ', 'ב', 'ר', 'א', ' ', 'א', 'ל', 'ה', 'י', 'ם', ' ', 'א', 'ת', ' ', 'ה', 'ש', 'מ', 'י', 'ם', ' ', 'ו', 'א', 'ת', ' ', 'ה', 'א', 'ר', 'ץ', ' ', 'ו', 'ה', 'א', 'ר', 'ץ', ' ', 'ה', 'י', 'ת', 'ה', ' ', 'ת', 'ה', 'ו', ' ', 'ו', 'ב', 'ה', 'ו', ' ', 'ו', 'ח', 'ש', 'ך', ' ', 'ע', 'ל', ' ', 'פ', 'נ', 'י', ' ', 'ת', 'ה', 'ו', 'ם', ' ', 'ו', 'ר', 'ו', 'ח', ' ', 'א', 'ל', 'ה', 'י', 'ם', ' ', 'מ', 'ר', 'ח', 'פ', 'ת', ' ', 'ע', 'ל', ' ', 'פ', 'נ', 'י', ' ', 'ה', 'מ', 'י']
12
[ 0  2 25  1 26 10 27  0  2 25 

array([ 0,  1,  6,  0,  5,  0,  0,  0,  9,  9,  0,  0,  2, 10,  5,  0,  0,
        0,  6,  0,  0,  8,  9,  8,  5,  0,  0,  1,  6,  0,  0,  9,  9,  7,
        0,  0,  1,  9,  9,  7,  0,  0,  9,  1,  9,  0,  0, 10,  0,  0,  0,
        9, 10,  0,  0,  0,  1, 10,  7,  1,  0,  8,  0,  0,  1,  6,  0,  0,
        1,  0, 10,  0,  0,  1,  0,  0,  8,  0,  2, 10,  5,  0,  0,  0,  1,
        8,  7,  7,  0,  0,  8,  0,  0,  1,  6,  0,  0,  8,  9,  5,  0,  0,
        8, 10,  0,  7,  0,  0,  2, 10,  5,  0,  0,  0,  1,  5,  0,  0,  0,
       10], dtype=int32)

In [159]:
num_letters = max(X)
num_pointings = [max(y) for y in ys]
num_letters, num_pointings

(27, [1, 2, 16])

In [224]:
seqlen = 60
def create_graph(seqlen=seqlen, insize=num_letters, outsize=num_pointings[2]+1, nhidden=64):
  graph = tf.Graph()
  with graph.as_default():
    
    X = tf.placeholder(tf.int64, shape=(None, seqlen), name='X')
    y = tf.placeholder(tf.int64, shape=(None, seqlen), name='y')
    
    X_onehot = tf.one_hot(X, num_letters, name='onehot')
    
    cell_fw = tf.nn.rnn_cell.LSTMCell(nhidden)
    cell_bw = tf.nn.rnn_cell.LSTMCell(nhidden)
    
    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, X_onehot, dtype=tf.float32)
    print(cell_fw.weights)
    outputs = tf.concat(outputs, axis=-1)
    
    dense = tf.layers.Dense(outsize)
    
    logits = dense(outputs)
    print(dense.weights)
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')
    
    lr = tf.placeholder(tf.float32, shape=(), name='lr')
    trn_op = tf.train.RMSPropOptimizer(lr, momentum=0.7, name='trn_op').minimize(loss)
    
    softmax = tf.nn.softmax(logits)
    predicts = tf.argmax(softmax, axis=-1, name='predictions')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicts, y), tf.float64), name='accuracy')
    
    init = tf.global_variables_initializer()
    
    tf.add_to_collections(['train feeds', 'test feeds'], X)
    tf.add_to_collections(['train feeds', 'test feeds'], y)
    tf.add_to_collection('train feeds', lr)
    
    tf.add_to_collection('fetches', trn_op)
    tf.add_to_collection('fetches', loss)
    tf.add_to_collection('fetches', accuracy)
  
  return graph
graph = create_graph()


[<tf.Variable 'bidirectional_rnn/fw/lstm_cell/kernel:0' shape=(91, 256) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/fw/lstm_cell/bias:0' shape=(256,) dtype=float32_ref>]
[<tf.Variable 'dense/kernel:0' shape=(128, 17) dtype=float32_ref>, <tf.Variable 'dense/bias:0' shape=(17,) dtype=float32_ref>]


In [0]:
def create_graph_with_numcheck(seqlen=seqlen, insize=num_letters, outsize=num_pointings[2], nhidden=64):
  graph = tf.Graph()
  with graph.as_default():
    
    X = tf.placeholder(tf.int64, shape=(None, seqlen), name='X')
    y = tf.placeholder(tf.int64, shape=(None, seqlen), name='y')
    
    X_onehot = tf.one_hot(X, num_letters, name='onehot')
    
    cell_fw = tf.nn.rnn_cell.LSTMCell(nhidden)
    cell_bw = tf.nn.rnn_cell.LSTMCell(nhidden)
    
    outputs, states = tf.check_numerics(tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, X_onehot, dtype=tf.float32))
    print(cell_fw.weights)
    outputs = tf.concat(outputs, axis=-1)
    
    dense = tf.layers.Dense(outsize)
    
    logits = dense(outputs)
    print(dense.weights)
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')
    
    lr = tf.placeholder(tf.float32, shape=(), name='lr')
    trn_op = tf.train.RMSPropOptimizer(lr, momentum=0.7, name='trn_op').minimize(loss)
    
    softmax = tf.nn.softmax(logits)
    predicts = tf.argmax(softmax, axis=-1, name='predictions')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicts, y), tf.float64), name='accuracy')
    
    init = tf.global_variables_initializer()
    
    tf.add_to_collections(['train feeds', 'test feeds'], X)
    tf.add_to_collections(['train feeds', 'test feeds'], y)
    tf.add_to_collection('train feeds', lr)
    
    tf.add_to_collection('fetches', trn_op)
    tf.add_to_collection('fetches', loss)
    tf.add_to_collection('fetches', accuracy)
  
  return graph
graph = create_graph()

In [217]:
def batch_gen(bs=64, X=X, y=ys[2], seqlen=seqlen, step=9):
  N = len(X)
  i = 0
  while step*(i+1)*bs+seqlen <= N:
    x_batch = [X[step*(i*bs+j):step*(i*bs+j)+seqlen] for j in range(bs)]
    y_batch = [y[step*(i*bs+j):step*(i*bs+j)+seqlen] for j in range(bs)]
    yield i, x_batch, y_batch
    i += 1
print(X[:10])
print(ys[2][:10])

[ 0  2 25  1 26 10 27  0  2 25]
[0 1 6 0 5 0 0 0 9 9]


In [100]:
graph = create_graph()
with graph.as_default():
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    gen = batch_gen(X=X, y=ys[2])
    for _ in range(10):
      _, bx, by = next(gen)
      feed = {graph.get_tensor_by_name('X:0'):bx, graph.get_tensor_by_name('y:0'):by}
      print(sess.run(graph.get_tensor_by_name('loss:0'), feed_dict = feed))

2.761125
2.7642653
2.7586205
2.7551486
2.7621267
2.7657428
2.7588878
2.7640033
2.762014
2.7645266


In [220]:
max(ys[2])

16

In [131]:
def find_lr(graph, batch_gen, start=1e-6, end=1e1, fac=2.0):
  lr = start
  lrs, losses = [], []
  with graph.as_default():
    print(tf.get_collection('feeds'))
    with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      while lr<end:
        _, bx, by = next(batch_gen)
        feed = dict(zip(tf.get_collection('feeds'), [bx, by, lr]))
        fetches = tf.get_collection('fetches')
        _, loss, acc = sess.run(fetches, feed_dict=feed)
        print(lr, loss, acc)
        lr *= fac

find_lr(create_graph(), batch_gen(X=X, y=ys[2]))

[<tf.Tensor 'X:0' shape=(?, 60) dtype=int64>, <tf.Tensor 'y:0' shape=(?, 60) dtype=int64>, <tf.Tensor 'lr:0' shape=() dtype=float32>]
1e-06 2.7692735 0.11223958333333334
2e-06 2.7697551 0.08619791666666667
4e-06 2.771639 0.10260416666666666
8e-06 2.780276 0.071875
1.6e-05 2.7745478 0.07578125
3.2e-05 2.777519 0.05859375
6.4e-05 2.77338 0.07109375
0.000128 2.7743814 0.058333333333333334
0.000256 2.7728999 0.08229166666666667
0.000512 2.7742593 0.06927083333333334
0.001024 2.7752347 0.06536458333333334
0.002048 2.7741735 0.06302083333333333
0.004096 2.7734103 0.07578125
0.008192 2.7669008 0.11223958333333334
0.016384 2.7531993 0.14739583333333334
0.032768 2.7226083 0.32005208333333335
0.065536 2.6842015 0.46979166666666666
0.131072 2.5872757 0.4796875
0.262144 2.380978 0.49140625
0.524288 2.0110328 0.4859375
1.048576 2.0723639 0.46979166666666666
2.097152 3.0380487 0.1015625
4.194304 2.8138301 0.0734375
8.388608 16.797659 0.5015625


In [227]:
def train_epoch(graph, gen, lr, epochs):
  with graph.as_default():
    with tf.Session() as sess:
      tf.summary.FileWriter('/log', graph=graph, session=sess)
      sess.run(tf.global_variables_initializer())
      for ie in range(epochs):
        ac_loss, ac_acc = 0, 0
        for ib, bx, by in gen():
          feed = dict(zip(tf.get_collection('train feeds'), [bx, by, lr]))
          _, loss, acc = sess.run(tf.get_collection('fetches'), feed_dict=feed)
          ac_loss += loss
          ac_acc += acc
          if ib%100==0 and ib>0:
            print("epoch {}, batch {} - loss: {}, accuracy: {}".format(ie, ib, ac_loss/100, ac_acc/100))
            ac_loss, ac_acc = 0, 0

train_epoch(create_graph(), lambda: batch_gen(bs=256), 0.1, 3)


[<tf.Variable 'bidirectional_rnn/fw/lstm_cell/kernel:0' shape=(91, 256) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/fw/lstm_cell/bias:0' shape=(256,) dtype=float32_ref>]
[<tf.Variable 'dense/kernel:0' shape=(128, 17) dtype=float32_ref>, <tf.Variable 'dense/bias:0' shape=(17,) dtype=float32_ref>]
epoch 0, batch 100 - loss: 1.5387536084651947, accuracy: 0.5350065104166665
epoch 0, batch 200 - loss: 0.7017419761419297, accuracy: 0.7634205729166671
epoch 0, batch 300 - loss: 0.5886567836999893, accuracy: 0.8021699218750001
epoch 0, batch 400 - loss: 0.5599243578314781, accuracy: 0.8138424479166666
epoch 1, batch 100 - loss: 0.5764878943562508, accuracy: 0.8179550781250002
epoch 1, batch 200 - loss: 0.5260412582755088, accuracy: 0.8293450520833333
epoch 1, batch 300 - loss: 0.547517860531807, accuracy: 0.815047526041667
epoch 1, batch 400 - loss: 0.529580470919609, accuracy: 0.8223255208333333
epoch 2, batch 100 - loss: 0.5458904957771301, accuracy: 0.8286822916666668
epoch 2, batch

In [0]:
def test(graph, gen):
  

In [164]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip



--2018-07-11 10:37:51--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.1.117.85, 34.206.253.53, 52.20.145.121, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.1.117.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  7.61MB/s    in 0.7s    

2018-07-11 10:37:53 (7.61 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

In [167]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://e04d930f.ngrok.io
